In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\FraudDetectionData.csv")

In [3]:
# Change the data type of column 'A' from float64 to float32
df['amount'] = df['amount'].astype('float32')
df['oldbalanceOrg'] = df['oldbalanceOrg'].astype('float32')
df['oldbalanceDest'] = df['oldbalanceDest'].astype('float32')
df['newbalanceOrig'] = df['newbalanceOrig'].astype('float32')
df['newbalanceDest'] = df['newbalanceDest'].astype('float32')

In [4]:
df['step'] = df['step'].astype('int32')
df['isFlaggedFraud'] = df['isFlaggedFraud'].astype('int32') 
df['isFraud'] = df['isFraud'].astype('int32') 

In [5]:
# Importing LabelEncoder from Sklearn
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder
# Creating a instance of label Encoder.
le = LabelEncoder()
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df['type'])
# printing label
label
# removing the column 'type' from df
# as it is of no use now.
df.drop("type", axis=1, inplace=True)
# Appending the array to our dataFrame
# with column name 'type'
df["type"] = label
# printing Dataframe
#df

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['nameDest'])
label
df.drop("nameDest", axis=1, inplace=True)
df["nameDest"] = label
#df

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['nameOrig'])
label
df.drop("nameOrig", axis=1, inplace=True)
df["nameOrig"] = label
#df

## Split

In [8]:
from sklearn.model_selection import train_test_split

X = df.drop('isFraud', axis=1)
# Separate the target variable
y = df['isFraud']

# Print class distribution before split
print(y.value_counts(normalize=True))

from sklearn.model_selection import train_test_split

# Assuming X contains your features and y contains your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=18)

# Print class distribution after split
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

0    0.998709
1    0.001291
Name: isFraud, dtype: float64
0    0.998709
1    0.001291
Name: isFraud, dtype: float64
0    0.99871
1    0.00129
Name: isFraud, dtype: float64


In [9]:
# Change the data type of column 'A' from float64 to float32
X_train['amount'] = X_train['amount'].astype('float32')
X_train['oldbalanceOrg'] =X_train['oldbalanceOrg'].astype('float32')
X_train['oldbalanceDest'] = X_train['oldbalanceDest'].astype('float32')
X_train['newbalanceOrig'] = X_train['newbalanceOrig'].astype('float32')
X_train['newbalanceDest'] = X_train['newbalanceDest'].astype('float32')

In [10]:
X_train['step'] = X_train['step'].astype('int32')
X_train['isFlaggedFraud'] = X_train['isFlaggedFraud'].astype('int32') 

## Class imbalance

In [11]:
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import ADASYN

# # Apply ADASYN only on the minority class of the train set
# ada = ADASYN()
# X_train_resampled, y_train_resampled = ada.fit_resample(X_train, y_train)

## Tomek's link- Remove noise

In [ ]:
# from imblearn.under_sampling import TomekLinks

# # Assuming X_train is your feature matrix and y_train is your target vector
# # and that the minority class is labeled as 1 and the majority class is labeled as 0
# tomek_links = TomekLinks(sampling_strategy='auto')
# X_train_resampled_new, y_train_resampled_new = tomek_links.fit_resample(X_train, y_train)

## Tomek followed by ENN

In [15]:
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours,OneSidedSelection

# Assume X_train and y_train are the original training data
# resample using TomekLinks first
tomek_links = TomekLinks(sampling_strategy='auto')
X_train_resampled, y_train_resampled = tomek_links.fit_resample(X_train, y_train)

# resample the output of TomekLinks using EditedNearestNeighbours
enn = EditedNearestNeighbours(sampling_strategy='auto')
X_train_resampled_new, y_train_resampled_new = enn.fit_resample(X_train_resampled, y_train_resampled)

In [16]:
X_train_resampled_new.shape

(5718840, 10)

In [17]:
import numpy as np

# assuming y_train is a numpy array or a pandas series
counts = np.bincount(y_train)
print("Class 0 count:", counts[0])
print("Class 1 count:", counts[1])

Class 0 count: 5718966
Class 1 count: 7392


In [19]:
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours,OneSidedSelection
# resample the output of EditedNearestNeighbours using One-Sided Selection
oss = OneSidedSelection(sampling_strategy='auto')
X_train_resampled_final, y_train_resampled_final = oss.fit_resample(X_train_resampled_new, y_train_resampled_new)

In [20]:
X_train_resampled_final.shape

(5713312, 10)

In [21]:
import numpy as np

# assuming y_train is a numpy array or a pandas series
counts = np.bincount(y_train)
print("Class 0 count:", counts[0])
print("Class 1 count:", counts[1])

Class 0 count: 5718966
Class 1 count: 7392


## Outliers

In [24]:
import random
random.seed (20)
import numpy as np
from scipy import stats

# Specify columns with outliers
cols_with_outliers = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

# Specify the number of bootstrapped samples to create per column
num_samples = 50

# Specify the right trimming proportions for each column
trim_props = {'amount': 0.14, 'oldbalanceOrg': 0.24, 'newbalanceOrig': 0.25, 'oldbalanceDest': 0.22, 'newbalanceDest': 0.22}

# Initialize empty dictionaries to store the trimmed means for each column
train_trimmed_means = {}

# Loop over the specified columns
for col_name in cols_with_outliers:
    
    # Check if the trimming proportion for this column is 0
    if trim_props[col_name] == 0:
        # If so, skip this column and move on to the next one
        continue
    
    # Initialize empty lists to store the bootstrapped samples and the trimmed means for the training set
    train_bootstrapped_samples = []
    train_trimmed_means_list = []
    
    # Loop over the number of desired samples
    for i in range(num_samples):
        # Randomly select indices from the column in the training set
        train_sample_indices = np.random.choice(X_train_resampled_final.index, size=len(X_train_resampled_final), replace=True)
        
        # Create a bootstrapped sample by indexing into the column with the selected indices for the training set
        train_sample = X_train_resampled_final.loc[train_sample_indices, col_name]
        
        # Append the bootstrapped samples to the list for the training set
        train_bootstrapped_samples.append(train_sample)
        
        # Calculate the right trimmed mean of the bootstrapped sample for the training set
        train_right_trimmed_mean = np.mean(train_sample[train_sample <= np.percentile(train_sample, 100*(1-trim_props[col_name]))])
        train_trimmed_means_list.append(train_right_trimmed_mean)
        
    # Calculate the mean of the right trimmed means for the training set and add it to the dictionary
    train_trimmed_means[col_name] = np.mean(train_trimmed_means_list)

    # Replace the outliers in the training set with the trimmed means
    X_train_resampled_final.loc[X_train_resampled_final[col_name] > np.percentile(X_train_resampled_final[col_name], 100*(1-trim_props[col_name])), col_name] = train_trimmed_means[col_name]

# Print the trimmed means for each column separately for the training set
print("Train set trimmed means: ", train_trimmed_means)

Train set trimmed means:  {'amount': 84062.25, 'oldbalanceOrg': 17587.316, 'newbalanceOrig': 12279.772, 'oldbalanceDest': 191113.31, 'newbalanceDest': 244841.23}


In [ ]:
import numpy as np
from sklearn.linear_model import Perceptron

# Create a Perceptron object
clf = Perceptron(random_state=0)

# Train the Perceptron on the data
clf.fit(X, y)

# Predict the output classes for the data points
y_pred = clf.predict(X)

# Check if the Perceptron correctly classified all the data points
if np.all(y_pred == y):
    print("Data is linearly separable")
else:
    print("Data is not linearly separable")

In [25]:
# Assuming your original training data is stored in a pandas DataFrame called X_train
# And assuming you have a list of selected feature names called selected_features
selected_features = ['oldbalanceOrg', 'type', 'nameDest','amount','step']
X_train_selected = X_train_resampled_final[selected_features]

In [ ]:
X_test_selected=X_test[selected_features]

## Create kernel

In [ ]:
import GPy
from IPython.display import display

we use kerne function to create covariance matrix

In [ ]:
#Create a 1-D kernel with default parameters
k=GPy.kern.RBF(1)
#Preview of parameters
k

## Visualize kernel

In [ ]:
x=np.linspace(-4.,4.,100)[:,None] # we need none to reshape x into a column vector to use in Gaussian

#first, sample kernel at x(dash)=0
K=k.K(x,np.array([[0.]])) # k(x,0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x,K)
plt.title("$\kappa_{rbf}{x,x'}$");

## Lengthscale

In [ ]:
fig, ax = plt.subplots()
from matplotlib.animation import FuncAnimation
from matplotlib import rc
ls = [0.05, 0.25, 0.5, 1., 2., 4]

def update(iteration):
    ax.cla()
    k = GPy.kern.RBF(1)
    k. lengthscale = ls[iteration]
    # Calculate the new covariance function at k(x,0)
    C = k.K(x, np.array([[0.]]))
    # Plot the resulting covariance vector
    ax.plot(x,C)
    ax.set_title(f"$\kappa_{{rbf}}(x,x')$ Length scale = {k.lengthscale[0]}")
    ax.set_ylim((0, 1.2))

num_iterations = len(ls)
anim = FuncAnimation(fig, update, frames=np.arange(0, num_iterations-1, 1), interval=500)
plt.close()

rc('animation', html='jshtml')
anim

## Variance

In [ ]:
fig, ax = plt.subplots()
from matplotlib.animation import FuncAnimation
from matplotlib import rc
variances = [0.05, 0.25, 0.5, 1., 2., 4]

def update(iteration):
    ax.cla()
    k = GPy.kern.RBF(1)
    k. variance = variances[iteration]
    # Calculate the new covariance function at k(x,0)
    C = k.K(x, np.array([[0.]]))
    # Plot the resulting covariance vector
    ax.plot(x,C)
    ax.set_title(f"$\kappa_{{rbf}}(x,x')$ Variance = {k.variance[0]}")
    ax.set_ylim((0, 1.2))

num_iterations = len(ls)
anim = FuncAnimation(fig, update, frames=np.arange(0, num_iterations-1, 1), interval=500)
plt.close()

rc('animation', html='jshtml')
anim

## Kernel function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF

# Define the RBF kernel with gamma=1.0
kernel = 1.0 * RBF(length_scale=1.0)

# Create a grid of points to evaluate the kernel function
x = np.linspace(-5, 5, 100)
X = np.atleast_2d(x).T

# Compute the kernel matrix
K = kernel(X)

# Plot the kernel function
plt.figure()
plt.plot(x, K[0], label='RBF kernel')
plt.xlabel('x')
plt.ylabel('k(x, 0)')
plt.legend(loc='best')
plt.show()

## Hyperparameter tuning for Gaussian distribution

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
# Define your reduced hyperparameter search space
param_grid = {
    'alpha': [0.1, 1, 5],
    'n_restarts_optimizer': [1, 2],
    'normalize_y': [True, False]
}

# Create an instance of the GaussianProcessRegressor
gp = GaussianProcessRegressor()

In [28]:
# Set up the GridSearchCV
search = GridSearchCV(gp, param_grid, cv=3, verbose=1, 
                      scoring='neg_mean_squared_error', refit=True)

In [30]:
# Fit the GridSearchCV object to the data
search.fit(X_train_selected, y_train_resampled_final)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


C:\Users\23059\anaconda3\envs\test2\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
24 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\23059\anaconda3\envs\test2\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\23059\anaconda3\envs\test2\lib\site-packages\sklearn\gaussian_process\_gpr.py", line 301, in fit
    self.kernel_.theta, clone_kernel=False
  File "C:\Users\23059\anaconda3\envs\test2\lib\site-packages\sklearn\gaussian_process\_gpr.py", line 529, in log_marginal_likeliho

MemoryError: Unable to allocate 238. TiB for an array with shape (5713312, 5713312) and data type float64

In [ ]:
# Get the best hyperparameters and evaluate on the test set
best_params = search.best_params_
best_model = search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test_selected)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Best hyperparameters:", best_params)
print("Test set accuracy:", accuracy)

https://machinelearningmastery.com/gaussian-processes-for-classification-with-python/

## Train Gaussian Process Model

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt


# define a Gaussian process classifier with RBF kernel
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, optimizer=None, random_state=0)

# perform k-fold cross-validation and compute ROC curve
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
for i, (train, test) in enumerate(cv.split(X, y)):
    probas_ = gpc.fit(X[train], y[train]).predict_proba(X[test])
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    mean_tpr += np.interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

# plot the mean ROC curve
mean_tpr /= cv.get_n_splits(X, y)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = %0.2f)' % mean_auc, lw=2)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Random', alpha=.8)

# set plot parameters and display
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# evaluate the model on a test set
test_auc = np.mean(cross_val_score(gpc, X, y, cv=10, scoring='roc_auc'))
print('Test AUC:', test_auc)